In [ ]:
#API 호출 및 데이터 품질 검사

import os
import requests
import json
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv() #코랩을 사용해서 추후 env형식으로 합쳐보겠습니다.

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

# API 호출 함수
API_URL = "https://api.example.com/disaster-info"

def fetch_disaster_data():
    params = {"key": api_key, "type": "json", "count": 100}
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        # 데이터 품질 검사
        clean_data = [item for item in data if item.get("title") and item.get("description")]
        print(f"Valid entries: {len(clean_data)} / {len(data)}")
        return clean_data
    else:
        print("Error fetching data:", response.status_code)
        return None

# 데이터 저장
data = fetch_disaster_data()
if data:
    with open("disaster_data.json", "w") as f:
        json.dump(data, f, indent=4)



In [ ]:
#pdf 로드 및 텍스트 추출

from langchain.document_loaders import PyPDFLoader

# PDF 로드 및 텍스트 추출
def load_pdf_with_pypdfloader(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    # 문서 리스트를 하나의 텍스트로 합치기
    full_text = "\n".join([doc.page_content for doc in documents])
    return documents, full_text

# PDF 로드 실행
pdf_path = ""  # PDF 경로
documents, full_text = load_pdf_with_pypdfloader(pdf_path)

# 텍스트 추출 확인
print(f"Extracted {len(documents)} pages of content.")
print(full_text[:500])  # 처음 500자 미리보기


In [ ]:
#텍스트 청킹 및 품질검사

from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

# 텍스트 분할기
def chunk_text_with_langchain(full_text, chunk_size=500, overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap, separators=["\n", " ", ""]
    )
    chunks = splitter.split_text(full_text)
    return chunks

# 텍스트 청킹 실행
chunks = chunk_text_with_langchain(full_text)

# 청크의 품질 검사: 최소 길이 기준 필터링
def quality_check(chunks, min_length=50):
    return [chunk for chunk in chunks if len(chunk) >= min_length]

filtered_chunks = quality_check(chunks)
print(f"Chunks after quality check: {len(filtered_chunks)} / {len(chunks)}")

# 텍스트 및 청크 저장
with open("filtered_chunks.json", "w") as f:
    json.dump(filtered_chunks, f, indent=4)


In [ ]:
#텍스트 임베딩

import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("API Key not found. Please set OPENAI_API_KEY in your environment.")

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 로컬 파일 저장소 설정
store = LocalFileStore("F:\\STUDY\\sparta\\999\\박성규\\emb")

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 모델 이름을 네임스페이스로 사용
)

# 예시 텍스트
texts = [
    "지진 발생 시 대처 방법은 무엇인가요?",
    "화재 발생 시 대피소 안내",
    "전쟁 상황에서의 시민 대피 방법"
]

# 텍스트 임베딩 생성
embeddings_result = cached_embedder.embed_documents(texts)

# 생성된 임베딩 확인
print("Generated embeddings:", embeddings_result)

# 캐시된 임베딩을 불러오는 예시
cached_result = store.load_data()  # 로컬 저장소에서 불러오기
print("Cached embeddings:", cached_result)


In [ ]:
#LLM retrieval

from sklearn.metrics.pairwise import cosine_similarity

# 유사 문서 검색 함수
def find_most_relevant_chunk(question, embeddings, chunks):
    question_embedding = model.encode([question])
    similarities = cosine_similarity(question_embedding, embeddings)[0]
    most_similar_index = np.argmax(similarities)
    return chunks[most_similar_index]

# 예시 질문
question = "지진 발생 시 대처 방안은?"
relevant_chunk = find_most_relevant_chunk(question, embeddings, filtered_chunks)
print("Relevant chunk:", relevant_chunk)




In [ ]:
#RAG

import openai

# OpenAI API 키 설정 (환경 변수에서 불러오기)
openai.api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 임베딩 모델 초기화
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=openai.api_key)

# RAG 방식 응답 생성
def generate_response_with_rag(question, relevant_text):
    system_prompt = (
        "You are an assistant that provides answers based on disaster manuals. "
        "Use the provided context to answer the question accurately."
    )

    # LLM 호출
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Context: {relevant_text}"},
            {"role": "user", "content": question}
        ]
    )
    return response["choices"][0]["message"]["content"]

# RAG 기반 응답 생성
response = generate_response_with_rag(question, relevant_chunk)
print("Answer:", response)


In [ ]:
# 테스트용 질문
questions = [
    "가뭄에 대처하는 방법은?",
    "전쟁 발생 시 시민들이 어떻게 대피해야 하나요?",
    "화재 발생 시 가장 중요한 행동은 무엇인가요?"
]

# 프롬프트 튜닝 및 테스트
for q in questions:
    relevant_text = find_most_relevant_chunk(q, embeddings, filtered_chunks)
    response = generate_response_with_rag(q, relevant_text)
    print(f"Question: {q}")
    print(f"Answer: {response}\n")
